In [ ]:
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16  # Example pre-trained image encoder
from tensorflow.keras.preprocessing.sequence import pad_sequences  # For padding captions
from keras.preprocessing.text import Tokenizer  # Example tokenizer

# Replace with paths to your model and tokenizer files
model_path = "path/to/your/image_captioning_model.h5"
tokenizer_path = "path/to/your/tokenizer.pkl"

# Load the model and tokenizer
model = tf.keras.models.load_model(model_path)
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

def preprocess_image(image_bytes):
    # Read image from bytes
    image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
    # Preprocess the image (resize) based on model requirements
    image = cv2.resize(image, (224, 224))  # Example for VGG16
    image = image.astype('float32') / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def generate_caption(image):
    # Extract features from the image using the pre-trained encoder
    features = VGG16(weights='imagenet', include_top=False).predict(image)
    # Start caption with the start token (adjust based on your tokenizer)
    start_token = tokenizer.word_index['<start>']
    predicted_caption = [start_token]
    max_length = 50  # Maximum caption length
    for _ in range(max_length):
        sequence = pad_sequences([predicted_caption], maxlen=len(predicted_caption), padding='post')
        prediction = model.predict(sequence)[0]
        predicted_word_index = np.argmax(prediction)
        predicted_word = tokenizer.index_word[predicted_word_index]
        if predicted_word == '<end>':
            break
        predicted_caption.append(predicted_word_index)
    caption = ' '.join(tokenizer.sequences_to_texts([predicted_caption]))[1:-1]  # Remove start/end tokens
    return caption

app = FastAPI()

@app.post("/upload")
async def upload_image(image: UploadFile = File(...)):
    # Read image content
    image_bytes = await image.read()
    # Preprocess image
    image = preprocess_image(image_bytes)
    # Generate caption
    caption = generate_caption(image)
    return {"caption": caption}

if __name__ == "__main__":
    import tensorflow as tf
    tf.config.experimental.set_memory_growth(True)  # Enable memory growth for GPU
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000)
